# Задача

Необходимо подобрать оптимальную геометрию замкового соединения лопатки турбины ГТД с диском.

В качестве исходных данных:
1. Объем лопатки "над" хвостовиком;
2. Плотность материала лопатки;
3. Центр тяжести лопатки без хвостовика;
4. Частота вращения ротора;
5. Ширина паза у верхнего зуба (исходя из геометрии лопатки);
6. Предельные значения напряжений в лопатке.

Как замковое соединение выглядит "в железе":

![Внешний вид замкового соединения, для общего представления](./Disk.jpg)

Схематичное изображение: 

![Внешний вид замкового соединения, для общего представления](./image047.png)

Обозначения геометрических параметров соединения:

![Геометрия замкового соединения](./geom.png)

# Основные формулы в виде функций

In [1]:
# Import necessary modules

import numpy as np
from numpy import sin, cos, tan
from scipy.optimize import NonlinearConstraint, Bounds, minimize, BFGS, SR1

In [3]:
# Coefficients for stresses calculations

# Buckling coefficient
def K_sm(x):
    gamma = x[0]
    beta = x[1]
    phi = x[2]
    psi = beta - (np.radians(90) - phi/2)
    m1 = (2 * sin(beta - gamma) / sin(gamma) - 0.7 * tan(1 / (gamma / 2)) - 0.05 / cos(psi))*cos(psi)
    return 1 / m1


# Bending coefficient
def K_iz(x):
    gamma = x[0]
    beta = x[1]
    phi = x[2]
    psi = beta - (np.radians(90) - phi/2)
    m1 = (2 * sin(beta - gamma) / sin(gamma) - 0.7 * tan(1 / (gamma / 2)) - 0.05 / cos(psi))*cos(psi)
    m4 = (2 * sin(beta - gamma) / sin(gamma) - 0.35 * tan(1 / (gamma / 2)) + 0.35 * tan(1 / (beta / 2))) * sin(gamma) / sin(beta - gamma)
    m5 = m1 / 2 / cos(psi) + 0.35 * tan(1 / (beta / 2)) - m4 / 2 * sin(beta - 90) + 0.05 / cos(psi)
    return 6 * m5 / m4**2 / cos(psi)


# Cut coefficient
def K_sr(x):
    gamma = x[0]
    beta = x[1]
    phi = x[2]
    psi = beta - (np.radians(90) - phi/2)
    m3 = (2 * sin(beta - gamma) / sin(gamma) - 0.35 * tan(1 / (gamma / 2)) - 0.05 / cos(psi))*sin(gamma) / sin(beta - gamma + phi / 2)
    return 1 / m3

# Fillet radius as function of S (joint step)
def r_func(S):
    return 0.1737 * S - 0.0146 # magic constants are taken by approximation of values from industry standard

# Fastener height
def h_f(z, S, phi): # z -- number of teeth pairs; phi in radians
    return cos(phi / 2) * S * (z - 0.5)


# This function is to find profile width (l_l) of the upper and bottom parts of fastener 
def l_l_first_last(z, S, r, gamma, beta, phi): # z -- number of teeth pairs
    l_d1 = S - 2 * (r / sin(gamma / 2) * cos(gamma / 2 - beta + np.radians(90) - phi / 2) - r) + \
        S * sin(beta - gamma) / sin(gamma) * cos(beta - np.radians(90) + phi / 2) + 8 * sin(phi / 2)
    double_delta = 0.05 * S
    l_l1 = l_d1 - double_delta
    
    l_last = l_l1 - 2 * sin(phi / 2) * S * (z - 0.5)
    return [l_l1, l_last]


# Fir-tree joint approximate volume
def vol_f(l_u, l_b, h, b): # _u -- upper section; _b -- bottom section; 
    S = (l_u + l_b) / 2 * h
    V = S * b
    R_cm = h / 3 * (2 * l_u +l_b) / (l_b + l_u) # http://ru.solverbook.com/question/centr-tyazhesti-trapecii/
    return  [V, R_cm]


def P_j(vol_u, vol_f, dens, n, R_u, R_f):    # vol_u -- volume of blade above fastener upper section; 
    vol = vol_u + vol_f                      # vol_f -- volume of fastener; dens -- material density; 
    omega = 2 * np.pi * n / 60               # n = rot. speed [rpm]; R_u --  upper blade part center of mass radius;
    dist = R_u - R_f                         # R_f -- fastener center of mass radius
    R = R_f + dist / (1 + vol_f / vol_u)     # dist -- distance between center of masses of blade and fastener
    return vol * dens * omega**2 * R         # R -- the whole blade center of mass


def l_h(l_l1, S, r, gamma, beta, phi):
    l_h = l_l1 - 2 * (S * sin(beta - gamma) / 2 / sin(gamma) - r / tan(gamma/2)) * \
            cos(beta + phi/2 - np.radians(90)) + 4 * r * sin(beta + phi/2 - np.radians(90)) - 4 * r
    return l_h

In [4]:
# Given values

# Blade center of mass without tail
blade_Rcm = 167.89 #mm

# Blade volume without tail
blade_vol = 3394.4957 # mm3

# Blade material density
mat_density = 8760 # kg / m3

# Rotation speed
n = 19200 # rpm

# Blade width 
l_d = 8.15 # mm

# Acceptable stresses
sigma_sm_dop =  230 # MPa 
sigma_iz_dop = 200
tau_sr_dop = 120

# Length of blade
b = 30 # mm

# Number of teeth (2...5)
z = 3

In [5]:
# Bounds for optimization values

# gamma, beta, phi, S
bounds = Bounds([np.radians(55), np.radians(105), np.radians(30), 1.8], [np.radians(65), np.radians(120), np.radians(50), 6.0])

In [3]:
np.radians(65)

1.1344640137963142

In [6]:
# Nonlinear constraints for optimizing function

def cons_func(x):
    gamma = x[0]
    beta = x[1]
    phi = x[2]
    S = x[3]
    psi = beta - (np.radians(90) - phi/2)
 
    l_upper = l_l_first_last(z, S, r_func(S), gamma, beta, phi)[0]
    l_lower = l_l_first_last(z, S, r_func(S), gamma, beta, phi)[1]
    fast_height = h_f(z, S, phi)
    fast_vol = vol_f(l_upper, l_lower, fast_height, b)
    CFF = P_j(blade_vol * 10**(-9), fast_vol[0] * 10**(-9), mat_density, n, blade_Rcm * 10**(-3), fast_vol[1] * 10**(-3)) / z / 2 # centrifugal force from the whole blade per 1 tooth

    m1 = (2 * sin(beta - gamma) / sin(gamma) - 0.7 * tan(gamma / 2)**(-1) - 0.05 / cos(psi))*cos(psi)
    m3 = (2 * sin(beta - gamma) / sin(gamma) - 0.35 * tan(gamma / 2)**(-1) - 0.05 / cos(psi))*sin(gamma) / sin(beta - gamma + phi / 2)
    m4 = (2 * sin(beta - gamma) / sin(gamma) - 0.35 * tan(1 / (gamma / 2)) + 0.35 * tan(1 / (beta / 2))) * sin(gamma) / sin(beta - gamma)
    m5 = m1 / 2 / cos(psi) + 0.35 * tan(beta / 2)**(-1) - m4 / 2 * sin(beta - 90) + 0.05 / cos(psi)
    sigma_sm = 1 / m1 * CFF / b / S * 2
    sigma_iz = 6 * m5 / m4**2 / cos(psi) * CFF / b / S * 2
    sigma_sr = 1 / m3 * CFF / b / S * 2
    return [sigma_sm, sigma_iz, sigma_sr]

In [7]:
nonlinear_constraint = NonlinearConstraint(cons_func, 0, [sigma_sm_dop, sigma_iz_dop, tau_sr_dop], jac = '2-point', hess = BFGS())

In [8]:
# Optimize

def optim_func(x):
    gamma = x[0]
    beta = x[1]
    phi = x[2]
    S = x[3]
 
    l_upper = l_l_first_last(z, S, r_func(S), gamma, beta, phi)[0]
    l_lower = l_l_first_last(z, S, r_func(S), gamma, beta, phi)[1]
    fast_height = h_f(z, S, phi)
    fast_vol = vol_f(l_upper, l_lower, fast_height, b)
    return fast_vol[0]

x0 = np.array([np.radians(20), np.radians(65), np.radians(115), 3.0])
res = minimize(optim_func, x0, method = 'trust-constr', jac = "2-point", hess = SR1(), constraints=nonlinear_constraint, options={'verbose': 1}, bounds=bounds)



`gtol` termination condition is satisfied.
Number of iterations: 124, function evaluations: 755, CG iterations: 138, optimality: 3.60e-10, constraint violation: 0.00e+00, execution time: 0.74 s.


In [9]:
print("gamma, beta, phi, S: {}".format(np.degrees(res.x)))
print(res.x)

print(cons_func(res.x))
print(optim_func(res.x))

gamma, beta, phi, S: [ 56.83904346 119.99926732  30.00033132 103.13255388]
[0.99202845 2.09438231 0.52360456 1.80000263]
[229.99721836734636, 60.07678395514359, 103.18186847464787]
439.2079117193173
